In [1]:
"""
MIT License

Copyright (c) 2023 Oscar Riveros

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [2]:
def oracle(sat, cnf):
    loc = len(cnf)
    for cls in cnf:
        for lit in cls:
            if lit > 0 and sat[abs(lit) - 1] == 1 or lit < 0 and sat[abs(lit) - 1] == 0:
                loc -= 1
                break
    return loc


def hess(num_variables, cnf):
    sat = [0] * num_variables
    opt = sat[:]
    cur = np.inf
    while True:
        done = True
        glb = np.inf
        for i in range(num_variables):
            sat[i] = 1 - sat[i]
            loc = oracle(sat, cnf)
            if loc < glb:
                glb = loc
                if glb < cur:
                    done = False
                    cur = glb
                    # print(glb)
                    opt = sat[:]
                    if cur == 0:
                        return opt
            elif loc > glb:
                sat[i] = 1 - sat[i]
        if done:
            break
    return opt

In [3]:
def generate_random_cnf_file(num_variables, num_clauses):
    cnf = []
    for _ in range(num_clauses):
        cls = []
        while len(cls) < np.random.randint(2, num_variables):
            var = np.random.randint(1, num_variables)
            if np.random.choice([True, False]):
                var = -var
            if not -var in cls and not var in cls:
                cls.append(var)
        cnf.append(cls)
    return cnf

In [4]:
def cnf_to_matrix(cnf, context_size):
    matrix = np.zeros(shape=(context_size, context_size))
    for i, cls in enumerate(cnf):
        for lit in cls:
            matrix[abs(lit) - 1][i] = -1 if lit < 0 else 1
    return np.asarray(matrix).flatten()

In [5]:
def normalize_assignemnt(assignemt, context_size):
    assignemt += (context_size - len(assignemt)) * [0]
    return np.asarray(assignemt).reshape(-1)

In [25]:
def algorithm_learning(algorithm, context_size, n_samples):
    N_HIDDEN = 128
    EPOCHS = 100
    VERBOSE = 1
    N_SAMPLES = n_samples
    BATCH_SIZE = 128
    DROPOUT = 0.1

    X_train, y_train = algorithm(context_size, N_SAMPLES)
    X_test, y_test = algorithm(context_size, N_SAMPLES) 

    model = tf.keras.models.Sequential()
    
    model.add(
        tf.keras.layers.Dense(units=N_HIDDEN,
                              input_shape=(context_size ** 2, ),
                              activation='relu'))
    
    model.add(
        tf.keras.layers.Dropout(DROPOUT)
    )

    model.add(
        tf.keras.layers.Dense(units=N_HIDDEN,
                              activation='relu'))
    
    model.add(
        tf.keras.layers.Dense(units=context_size,
                              activation='softmax'))
    
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    
    model.fit(
        x=X_train,
        y=y_train,
        epochs=EPOCHS,
        verbose=VERBOSE,
        batch_size=BATCH_SIZE   
    )

    _, test_acc = model.evaluate(
        X_test,
        y_test)
    
    print('test accuracy', test_acc)
    
    return model

In [26]:
def apply_hess(context_size, n_samples):
    N_VARIABLES = np.random.randint(1, context_size)
    N_CLAUSES = np.random.randint(1, context_size)
    inputs = []
    outputs = []
    for _ in range(n_samples):
        cnf = generate_random_cnf_file(N_VARIABLES, N_CLAUSES)
        matrix = cnf_to_matrix(cnf, context_size)
        optimal = hess(N_VARIABLES, cnf)
        assignemnt = normalize_assignemnt(optimal, context_size)
        inputs.append(matrix)
        outputs.append(assignemnt)
    return np.asarray(inputs), np.asarray(outputs)

In [27]:
CONTEXT_SIZE = 100
N_SAMPLES = 2000

hess_model = algorithm_learning(
    apply_hess, 
    context_size=CONTEXT_SIZE, 
    n_samples=N_SAMPLES)

Epoch 1/100
16/16 [==============================] - 1s 19ms/step - loss: 0.4139 - binary_accuracy: 0.8463
Epoch 2/100
16/16 [==============================] - 0s 7ms/step - loss: 0.2391 - binary_accuracy: 0.8540
Epoch 3/100
16/16 [==============================] - 0s 8ms/step - loss: 0.2164 - binary_accuracy: 0.8556
Epoch 4/100
16/16 [==============================] - 0s 8ms/step - loss: 0.1946 - binary_accuracy: 0.8553
Epoch 5/100
16/16 [==============================] - 0s 8ms/step - loss: 0.1646 - binary_accuracy: 0.8550
Epoch 6/100
16/16 [==============================] - 0s 9ms/step - loss: 0.1353 - binary_accuracy: 0.8544
Epoch 7/100
16/16 [==============================] - 0s 8ms/step - loss: 0.1140 - binary_accuracy: 0.8542
Epoch 8/100
16/16 [==============================] - 0s 9ms/step - loss: 0.0988 - binary_accuracy: 0.8544
Epoch 9/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0873 - binary_accuracy: 0.8545
Epoch 10/100
16/16 [=========================

In [24]:
tf.keras.models.save_model(hess_model, './checkpoints/hess_model_{}'.format(CONTEXT_SIZE))

INFO:tensorflow:Assets written to: ./checkpoints/hess_model_100\assets
